In [1]:
import os

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [4]:
#load the historical file and convert lines to rows
lines = sc.textFile("file:///Users/anjabelcijan/Desktop/spark/out2.txt")
words = lines.map(lambda l: l.split("\t"))

In [5]:
#delete first row
header = words.first()
words = words.filter(lambda line: line != header)

In [6]:
#create dataframe with columns
df1 = words.map(lambda p: Row(Title=p[0], Description=p[1], Topic=p[2], label=int(p[3])))
df = spark.createDataFrame(df1)
df.show(5)

+--------------------+--------------------+---------------+-----+
|         Description|               Title|          Topic|label|
+--------------------+--------------------+---------------+-----+
|"'I wasn't feelin...|"Meek Mill Wishes...|"entertainment"|    1|
| "Don't leak nudes."|"A Victim Of Reve...|"entertainment"|    1|
|   "It's wine time."|"Make A Pitcher O...|"entertainment"|    1|
|"Which tune will ...|"Create A Summer ...|"entertainment"|    1|
|"Are you a combo ...|"Everyone Is A Co...|"entertainment"|    1|
+--------------------+--------------------+---------------+-----+
only showing top 5 rows



In [7]:
#import stopwords
stopwords = sc.textFile("file:///Users/anjabelcijan/Desktop/spark/stopwords.txt").collect()

In [8]:
#import punctuation fn
import re
punctuation = re.compile(r'[-.?!,":;()|0-9]')

In [9]:
#define fn to extract tokens from words
def extract_tokens(sentence):
    """Tokenization of a given sentence.
       Drop stopwords, punctuations, numbers.
       Change the sentence to lowercase."""
    
    if(sentence) :
        tokens = punctuation.sub(' ', sentence.replace("'", " ")).lower().split()
        tokens_filtered = [word for word in tokens if (word not in stopwords) and (len(word) > 2)]
        return tokens_filtered
    else :
        return []

In [10]:
#test function
extract_tokens("Hello, World!")

['world']

In [11]:
# extract tokens
from pyspark.sql.functions import udf
from pyspark.sql.types import *

# Create UDF
extract_tokens_udf = udf(lambda sentence: extract_tokens(sentence), ArrayType(StringType()))

# Add tokens for title to DF
df_tokens = df.withColumn('tokens', extract_tokens_udf(df.Title))

In [13]:
# Add tokens for description to DF
df_tokens1 = df_tokens.withColumn('tokens2', extract_tokens_udf(df_tokens.Description))

In [14]:
df_tokens1.show(3)

+--------------------+--------------------+---------------+-----+--------------------+------------------+
|         Description|               Title|          Topic|label|              tokens|           tokens2|
+--------------------+--------------------+---------------+-----+--------------------+------------------+
|"'I wasn't feelin...|"Meek Mill Wishes...|"entertainment"|    1|[meek, mill, wish...|   [wasn, feeling]|
| "Don't leak nudes."|"A Victim Of Reve...|"entertainment"|    1|[victim, revenge,...|[don, leak, nudes]|
|   "It's wine time."|"Make A Pitcher O...|"entertainment"|    1|[make, pitcher, s...|      [wine, time]|
+--------------------+--------------------+---------------+-----+--------------------+------------------+
only showing top 3 rows



In [15]:
df_tokens1.dtypes

[('Description', 'string'),
 ('Title', 'string'),
 ('Topic', 'string'),
 ('label', 'bigint'),
 ('tokens', 'array<string>'),
 ('tokens2', 'array<string>')]

In [19]:
# apply hashing fn to tokens
from pyspark.ml.feature import HashingTF
hashingTF = HashingTF(inputCol="tokens", outputCol="features", numFeatures=100)
featurizedData = hashingTF.transform(df_tokens1)

Py4JJavaError: An error occurred while calling o178.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 7, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/anjabelcijan/Desktop/spark/spark-2.3.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 229, in main
    process()
  File "/Users/anjabelcijan/Desktop/spark/spark-2.3.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 224, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/anjabelcijan/Desktop/spark/spark-2.3.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-6-8bc3ee245033>", line 2, in <lambda>
IndexError: list index out of range

	at org.apache.spark.api.python.BasePy

In [20]:
hashingTF = HashingTF(inputCol="tokens2", outputCol="features2", numFeatures=100)
featurized2Data = hashingTF.transform(featurizedData)

featurized2Data.show(3)

+--------------------+--------------------+---------------+-----+--------------------+------------------+--------------------+--------------------+
|         Description|               Title|          Topic|label|              tokens|           tokens2|            features|           features2|
+--------------------+--------------------+---------------+-----+--------------------+------------------+--------------------+--------------------+
|"'I wasn't feelin...|"Meek Mill Wishes...|"entertainment"|    1|[meek, mill, wish...|   [wasn, feeling]|(100,[38,76,77,91...|(100,[29,62],[1.0...|
| "Don't leak nudes."|"A Victim Of Reve...|"entertainment"|    1|[victim, revenge,...|[don, leak, nudes]|(100,[9,41,57,70,...|(100,[4,21],[2.0,...|
|   "It's wine time."|"Make A Pitcher O...|"entertainment"|    1|[make, pitcher, s...|      [wine, time]|(100,[0,25,36,56,...|(100,[57,97],[1.0...|
+--------------------+--------------------+---------------+-----+--------------------+------------------+-------

In [84]:
#create vector out of raw features (not sure if useful)
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
pair_assembler = VectorAssembler(inputCols=["features", "features2"], outputCol="vector")
output = pair_assembler.transform(featurized2Data)

In [85]:
output.show(5)

+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               Title|         Description|        Topic|              tokens|             tokens2|             bigrams|            bigrams2|            features|           features2|              vector|
+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Meek Mill Wishes ...|'I wasn't feeling...|entertainment|[meek, mill, wish...|     [wasn, feeling]|[meek mill, mill ...|      [wasn feeling]|(100,[38,76,77,91...|(100,[29,62],[1.0...|(200,[38,76,77,91...|
|A Victim Of Reven...|   Don't leak nudes.|entertainment|[victim, revenge,...|  [don, leak, nudes]|[victim revenge, ...|[don leak, leak n...|(100,[9,41,57,70,...|(100,[4,21],[2.0,.

In [23]:
# set seed for reproducibility
(trainingData, testData) = featurized2Data.randomSplit([0.7, 0.3], seed = 100)